# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,shiqian,27.5225,108.2281,279.86,90,100,1.52,CN,1702329601
1,1,shakawe,-18.3667,21.8500,296.58,62,39,2.56,BW,1702329601
2,2,margaret river,-33.9500,115.0667,287.73,74,0,3.95,AU,1702329360
3,3,chonchi,-42.6128,-73.8092,286.71,94,100,2.02,CL,1702329414
4,4,blackmans bay,-43.0167,147.3167,287.22,89,21,0.45,AU,1702329603


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_center = [0, 0]  # Set the center of the map (e.g., [0, 0] for a world map)
folium_map = folium.Map(location=map_center, zoom_start=2)

# Add a marker for each city with the size of the point representing humidity
for index, row in city_data_df.iterrows():
    city_location = [row['Lat'], row['Lng']]
    humidity = row['Humidity']
    
    # Create a CircleMarker for each city with a popup displaying city information
    folium.CircleMarker(
        location=city_location,
        radius=humidity / 2,  # Adjust the radius size based on humidity
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"City: {row['City']}<br>Humidity: {humidity}%"
    ).add_to(folium_map)

# Save the map as an HTML file
folium_map.save("output_data/humidity_map.html")

# Display the map
folium_map


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# More permissive ideal weather conditions
ideal_conditions = {
    "Max Temp": (0, 100),   # Any temperature
    "Humidity": (0, 100),   # Any humidity level
    "Cloudiness": (0, 100), # Any cloudiness level
    "Wind Speed": (0, 100)  # Any wind speed
}

# Narrow down the DataFrame based on more permissive ideal conditions
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] >= ideal_conditions["Max Temp"][0]) &
    (city_data_df["Max Temp"] <= ideal_conditions["Max Temp"][1]) &
    (city_data_df["Humidity"] >= ideal_conditions["Humidity"][0]) &
    (city_data_df["Humidity"] <= ideal_conditions["Humidity"][1]) &
    (city_data_df["Cloudiness"] >= ideal_conditions["Cloudiness"][0]) &
    (city_data_df["Cloudiness"] <= ideal_conditions["Cloudiness"][1]) &
    (city_data_df["Wind Speed"] >= ideal_conditions["Wind Speed"][0]) &
    (city_data_df["Wind Speed"] <= ideal_conditions["Wind Speed"][1])
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
print(f"Number of cities meeting more permissive ideal conditions: {len(ideal_cities_df)}")
print(ideal_cities_df.head())

# Display any rows in the original dataset with null values
print("Rows with null values in the original dataset:")
print(city_data_df[city_data_df.isnull().any(axis=1)])
# Drop rows with null values in the original dataset
city_data_df = city_data_df.dropna()




Number of cities meeting more permissive ideal conditions: 0
Empty DataFrame
Columns: [City_ID, City, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Country, Date]
Index: []
Rows with null values in the original dataset:
     City_ID          City      Lat      Lng  Max Temp  Humidity  Cloudiness  \
93        93      khorixas -20.3667  14.9667    294.77        39          13   
139      139    oranjemund -28.5500  16.4333    289.49        92           1   
205      205  grootfontein -19.5667  18.1167    296.91        25          66   
212      212      luderitz -26.6481  15.1594    292.33        58           0   
249      249       omaruru -21.4333  15.9333    297.09        30          28   
336      336         opuwo -18.0607  13.8400    297.51        28          58   

     Wind Speed Country        Date  
93         3.98     NaN  1702329635  
139        6.00     NaN  1702329645  
205        2.06     NaN  1702329666  
212        9.39     NaN  1702329673  
249        2.93     N

### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = ideal_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# ... (Previous code)

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"distanceToPoint(location, {lng},{lat}) < {radius}"
    params["bias"] = f"proximity,{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Print API request URL for debugging
    print(f"API Request URL: {base_url}?{params}")
    
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Print API response for debugging
    print(f"API Response: {name_address}")
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# ... (Remaining code)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_center = [0, 0]  # Set the center of the map (e.g., [0, 0] for a world map)
folium_map = folium.Map(location=map_center, zoom_start=2)

# Add a marker for each city with the size of the point representing humidity
for index, row in hotel_df.iterrows():
    city = row['City']
    country = row['Country']
    hotel_name = row['Hotel Name']
    humidity = row['Humidity']
    lat = row['Lat']
    lng = row['Lng']

    # Create a popup message with additional information
    popup_msg = f"{city}, {country}<br>Hotel: {hotel_name}<br>Humidity: {humidity}%"

    # Add a marker to the map
    folium.Marker(
        location=[lat, lng],
        popup=popup_msg,
        icon=folium.Icon(color='blue')
    ).add_to(folium_map)

# Display the map
folium_map
